In [23]:
import random
import string
from Crypto.Util.number import isPrime, getPrime, inverse
from pwn import *

BLOCK_LEN = 129
CHARSET = string.ascii_uppercase + string.ascii_lowercase + string.digits
users, pwd_hashes = {}, []
allowed_blocks = []

q1 = 57895665874783536962369408363969823887021530656373208299565102620846005563716018275834077962292286213472570266375824572745671541793458387390711613089471407869558363212866932533545785125988453002675479793768261480181947144057144941974626043243654731721303589851520175899531854692118423229594279209070187162279
p1 = 2 * q1 + 1
g1 = 2
print(CHARSET, len(CHARSET))

ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789 62


In [3]:
# Test thử pass toàn 128 số 1
ad_pass = bytes([1] * (BLOCK_LEN - 1)) 
ad_pass = ad_pass.hex() + '00'
print(ad_pass)

print(bytes.fromhex(ad_pass))

010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010100
b'\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x00'


In [ ]:
# Lấy token admin
# 183.91.11.30 666
# Tạo một conection remote:
port = 666
r = remote('183.91.11.30', port)

# Lấy flag được mã hóa
get_success = r.recvuntil('> ').decode()
r.sendline('1337')
get_success = r.recvuntil('the victim name: ').decode()
r.sendline('admin')
get_success = r.recvuntil("don't tell anyone about this\n").decode()
token_admin = r.recvline(keepends=False).strip().decode()
token_admin = list(map(int, token_admin.strip('[]').split(',')))

# Đăng ký một user tên admin1:
get_success = r.recvuntil('> ').decode()
r.sendline('1')
get_success = r.recvuntil('Enter username: ').decode()
r.sendline('admin1')
get_success = r.recvuntil('Enter password: ').decode()
pass_test = ad_pass
r.sendline(pass_test)

# Lấy token của mã chèn vào
get_success = r.recvuntil('> ').decode()
r.sendline('1337')
get_success = r.recvuntil('the victim name: ').decode()
r.sendline('admin1')
get_success = r.recvuntil("don't tell anyone about this\n").decode()
token_admin1 = r.recvline(keepends=False).strip().decode()
token_admin1 = list(map(int, token_admin1.strip('[]').split(',')))


print(token_admin1)
print(len(token_admin1), type(token_admin1))
# r.close()

In [22]:
a = pow(sum(token_admin), 1, q1)
b = pow(sum(token_admin1), 1, q1)
def finds_x (a, b):
    for x in range(1, 256):
        for y in range(1, 256):
            if (a*x-b*y)%q1 == 0:
                print(x, y)
                return

# for i in range(0, 128):
#     for j in range(0,128):
#         finds_x(token_admin[i], token_admin1[j])


mina = min(token_admin)
minb = min(token_admin1)

print(mina//minb)

# print(a)
# print(b)

0


In [49]:
tests_token = token_admin

In [50]:
x1 = random.randint(1, 2 ** 256)
y1 = pow(g1, x1, p1)

# Test thử pass
ad_pass = ''.join(random.choices(CHARSET, k = BLOCK_LEN - 1)).encode() + b'\x00'
ad_pass.hex()

def secure_hash(data, token, is_login = False):
    assert len(data) + 1 >= BLOCK_LEN, "Invalid Length"
    
    if len(data) % BLOCK_LEN != 0:
        data += b'\x80'
        data += b'\x00' * (BLOCK_LEN - len(data) % BLOCK_LEN - 1)
    print(f"Data Pass: {data}")    
    blocks = [data[i:i + BLOCK_LEN] for i in range(0, len(data), BLOCK_LEN)]
    bases = [pow(g1, x, p1) for x in token] + [g1]
    yu_1 = y1 
    
    for block in blocks:
        # if is_login:
        #     if block not in allowed_blocks:
        #         print("Invalid block")
        #         return False
        yu_1 = block_hash(block, bases, yu_1)
        allowed_blocks.append(block)
        print(f"Block: {block}")
        print(f"Base: {bases}")
        print(f"x1 = {x1}")
        print(f"y1 = {y1}")
        print(f"yu1 = {yu_1}")
    return yu_1


tokens = tests_token
# Đưa vào hàm block_hash
def block_hash(block, bases, a):
    block_i = []
    for x, y in zip(bases, block):
        a = a * pow(x, y, p1) % p1
        block_i.append(pow(x, y, p1))
    print(f"Block_i: {block_i[-1]}")
    print(bases[-1], block[-1])
    return a

aa = secure_hash(ad_pass, tokens, True)
print(aa)

Data Pass: b'1Fmo6428muIjK52m1TQevORI0D6hJBome0iqwWRvhoshWbJy4VBFqEwfdHrScxej0DpqX8Vm2MxScWuW76fJKqjZxPA90DtgxRzuFqt5oShHVPUJrk3QdYyPmxFj9NX9\x00'
Block_i: 1
2 0
Block: b'1Fmo6428muIjK52m1TQevORI0D6hJBome0iqwWRvhoshWbJy4VBFqEwfdHrScxej0DpqX8Vm2MxScWuW76fJKqjZxPA90DtgxRzuFqt5oShHVPUJrk3QdYyPmxFj9NX9\x00'
Base: [..., 2]
x1 = 86658988690485426009714398314991611110407770549862719925636620960010589536843
y1 = 89906978927274236512244777206120863737674151845867922674424313131405180659074820240166255548204200398067992056566731843857349266887158920227539391601435036884340440396460626956569511545642477077370314004766844655573190758117812435329648754541381776973300321882819438853375835954299568050529553388871335989167
yu1 = 3290008970218413090549125562878672926990203154638230125070231741871936698726241233558758628480296128447961445288861453035891799722654728274208247748956636854348304827504473187582189580780832896072277477344565854988489795416324683375114895861133573145062852916830155395756894824

In [58]:
# Test thử pass
ad_pass = ''.join(random.choices(CHARSET, k = BLOCK_LEN - 1)).encode() 


print(ad_pass)
ad_pass = ad_pass.hex() + '00'
print(ad_pass)
print(bytes.fromhex(ad_pass))

b'4oJkg8FSjeJrAYx66FsASAb8B7oYGRvkWBMbTFIW3OO0Lxy7xxCYXOiekDJgRbQcqZZSChVRtq9i0NIq52SxnPWygk8NNrbGqhdadZupPYAP9sVwrzjSbg9rfKRuNnHd'
346f4a6b673846536a654a7241597836364673415341623842376f594752766b57424d6254464957334f4f304c78793778784359584f69656b444a6752625163715a5a534368565274713969304e4971353253786e505779676b384e4e72624771686461645a75705059415039735677727a6a5362673972664b52754e6e486400
b'4oJkg8FSjeJrAYx66FsASAb8B7oYGRvkWBMbTFIW3OO0Lxy7xxCYXOiekDJgRbQcqZZSChVRtq9i0NIq52SxnPWygk8NNrbGqhdadZupPYAP9sVwrzjSbg9rfKRuNnHd\x00'


In [72]:
# Test thử pass toàn 128 số 0
ad_pass = bytes([0] * (BLOCK_LEN - 2)) 
ad_pass = '01'+ ad_pass.hex()
print(ad_pass)
print(bytes.fromhex(ad_pass))


a = random.randint(1, q1 - 1)
b = random.randint(1, q1 - 1)

print(a)
print(b)

def find_x(a, b, q):
    mod = q - 1
    b_inv = inverse(b, mod)
    x = (a * b_inv) % mod
    return x
print(pow(2,q1,p1))


def finds_x (a, b):
    for x in range(0, 256):
        if (a*x-b)%q1 == 0:
            print(x)
            return


0100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
b'\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
12444846619691509790351622342188436115628891490635868192054164361572678480719361653492728582730627044315390209269280360380446057789221409085573276722348536445873659664334475420808314001635816206375874128698563270290547660578550

In [26]:
admin_password = ''.join(random.choices(CHARSET, k = BLOCK_LEN - 1))
print(admin_password)

QeBVRzuXb687It0nivqgvtI0OPgvHkLElqD1MQuTs2RFOTQNTrHTUiwDHSSJV1RuQ80TW4X8nU0rYN52scpELdoOlPlsCdLaWrZV2PbDBtC6oWWHgmRT6SpCtlXY5j9I
